In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import sklearn
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from math import radians, cos, sin, asin, sqrt

import os
print(os.listdir("../input"))


['train.csv', 'sample_submission.csv', 'test.csv']


# Import the train dataset 

In [2]:
pn = pd.read_csv('../input/train.csv', index_col='id') 
pn['pickup_datetime'] = pd.to_datetime(pn['pickup_datetime']) #i Convert the date  type as the  le date format 
pn['dropoff_datetime'] = pd.to_datetime(pn['dropoff_datetime']) #i Convert the date  type as the  le date format 
pn.head()


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [3]:
pn = pn[pn.passenger_count >= 1 ]
pn = pn[pn.trip_duration <= 7200]
pn = pn[pn.trip_duration >= 200]

In [4]:
pn['year_pickup'] = pn['pickup_datetime'].dt.year 
pn['month_pickup']=pn['pickup_datetime'].dt.month 
pn['day_pickup']=pn['pickup_datetime'].dt.day
pn['weekday_pickup'] = pn['pickup_datetime'].dt.weekday
pn['hour_pickup']=pn['pickup_datetime'].dt.hour

In [5]:
pn.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,year_pickup,month_pickup,day_pickup,weekday_pickup,hour_pickup
count,1.364202e+06,1.364202e+06,1.364202e+06,1.364202e+06,1.364202e+06,1.364202e+06,1.364202e+06,1364202.0,1.364202e+06,1.364202e+06,1.364202e+06,1.364202e+06
mean,1.535224e+00,1.667840e+00,-7.397350e+01,4.075052e+01,-7.397344e+01,4.075141e+01,8.833023e+02,2016.0,3.521378e+00,1.551488e+01,3.043861e+00,1.365909e+01
std,4.987579e-01,1.315066e+00,7.245138e-02,3.284911e-02,7.217761e-02,3.605625e-02,6.493284e+02,0.0,1.681146e+00,8.702559e+00,1.949116e+00,6.374469e+00
min,1.000000e+00,1.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,2.000000e+02,2016.0,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399200e+01,4.073697e+01,-7.399139e+01,4.073519e+01,4.430000e+02,2016.0,2.000000e+00,8.000000e+00,1.000000e+00,9.000000e+00
50%,2.000000e+00,1.000000e+00,-7.398192e+01,4.075359e+01,-7.397990e+01,4.075415e+01,7.010000e+02,2016.0,4.000000e+00,1.500000e+01,3.000000e+00,1.400000e+01
75%,2.000000e+00,2.000000e+00,-7.396784e+01,4.076789e+01,-7.396328e+01,4.076920e+01,1.112000e+03,2016.0,5.000000e+00,2.300000e+01,5.000000e+00,1.900000e+01
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,7.191000e+03,2016.0,6.000000e+00,3.100000e+01,6.000000e+00,2.300000e+01


In [6]:
selected_columns = ['passenger_count','pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
                   'dropoff_latitude', 'year_pickup',
                   'month_pickup', 'day_pickup', 'hour_pickup','weekday_pickup', 
                   ]
X = pn[selected_columns]
y = pn['trip_duration']
X.shape, y.shape

((1364202, 10), (1364202,))

# Cross&Validation 

In [7]:
cv = ShuffleSplit(1, test_size=0.01, train_size=0.02, random_state=0) 

In [8]:
from sklearn.metrics import mean_squared_error
rf = RandomForestRegressor()
scores = cross_val_score(rf, X , y , cv=cv ,scoring='neg_mean_squared_log_error')
np.sqrt(- scores.mean())


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.3691190073572256

In [9]:
rf.fit (X,y)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

# Import the test dataset 


In [10]:
pn_test = pd.read_csv('../input/test.csv', index_col='id') 
pn_test['pickup_datetime'] = pd.to_datetime(pn_test['pickup_datetime']) #i Convert the date  type as the  le date format 
pn_test.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
id,,,,,,,,
id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [11]:
pn_test['year_pickup'] = pn_test['pickup_datetime'].dt.year
pn_test['month_pickup']=pn_test['pickup_datetime'].dt.month 
pn_test['day_pickup']=pn_test['pickup_datetime'].dt.day
pn_test['weekday_pickup'] = pn_test['pickup_datetime'].dt.weekday
pn_test['hour_pickup']=pn_test['pickup_datetime'].dt.hour

In [12]:
pn_test.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,year_pickup,month_pickup,day_pickup,weekday_pickup,hour_pickup
id,,,,,,,,,,,,,
id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,2016,6,30,3,23
id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,2016,6,30,3,23
id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,2016,6,30,3,23
id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,2016,6,30,3,23
id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,2016,6,30,3,23


In [13]:
X_test = pn_test[selected_columns]


In [14]:
y_pred = rf.predict(X_test)
y_pred.mean()

861.7588242894057

# Import the sample_submission dataset   

In [15]:
sub = pd.read_csv('../input/sample_submission.csv')
sub.head()

,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959


In [16]:
sub['trip_duration'] = y_pred 
sub.head()

,id,trip_duration
0,id3004672,907.8
1,id3505355,778.1
2,id1217141,463.3
3,id2150126,1097.6
4,id1598245,415.7


In [17]:
sub.describe()


,trip_duration
count,625134.000000
mean,861.758824
std,584.606468
min,211.000000
25%,458.700000
50%,691.400000
75%,1071.700000
max,5987.300000


In [18]:
sub.to_csv('sub.csv', index=False)


In [19]:
!ls

__notebook_source__.ipynb  sub.csv
